# L'anneau des rationnels.

Ce cahier explore brièvement l'anneau des rationnels avec Python et Numpy. En informatique, les rationnels sont mélangés avec les nombres réels pour être représentés par ce qu'on appelle des nombres flottants. 

Nous allons construire dans ce cahier une autre solution qui aurait été possible: représenter un rationnel par un couple d'entiers, son numérateur et son dénominateur.

$$ \cal{M}: \quad q \in \mathbb{Q} \quad \longmapsto \quad (n , d) \in \mathbb{N}^2, \; \text{avec} \: d \neq 0$$

Ceci va nous permettre de comprendre l'application concrètre de nos structures algébriques et des morphismes entre structures algébrtiques.

Nous verrons à la fin pourquoi cette solution que nous exponsons n'a finalement pas été retenue.

## Première solution

Comme nous l'avons vu dans les rappels d'algèbre, la définition d'un objet (ici un couple d'entier représentant un rationel) n'est pas suffisante pour qualifier une structure. Il faut y adjoindre au moins une operation interne, celle qu'on appelle par défaut l'addition. Pour la définir, nous utilisons nos souvenirs sur l'addition des rationnels en donnant à chacun des couples un dénominateur commun. Voici une première solution simpliste de cette addition.

In [8]:
def add(x , y):
    """Première version de l'addition de deux rationnels représentés par des couples d'entiers."""
    return (x[0]*y[1] + y[0]*x[1], x[1]*y[1])

x1 = [2, 3]      #  = 2 / 3
y1 = [1, 3]      #  = 1 / 3
s1 = add(x1,y1)  #  = 2 / 3 + 1 / 3 = 1

print(f"Nous avons fait l'opération: ({x1} + {y1}) = {s1}")

Nous avons fait l'opération: ([2, 3] + [1, 3]) = (9, 9)


L'addition que nous avons définie est bien une opération interne sur l'ensemble des couples d'entiers. Elle est associative. Nous avons au moins un élément neutre comme le couple $(0,1)$ ou tous les couples dont le numérateur est nul. Tout couple à un symétrique : il suffit de prendre l'opposé du numérateur. Enfin cette opération est aussi commutative.

En conclusion, l'ensemble des couples d'entiers muni de l'addition que avons définie est un groupe commutatif.

On peut aussi vérifier que l'association que nous avons faite entre l'ensemble des rationnels muni de l'addition et l'ensemble des couples d'entiers avec l'addition que nous avons définie est un morphisme de groupe.

$
\begin{array}{llrl}
\cal{M}: & q1 & \mapsto & (n1,d1) \\
         & q2 & \mapsto & (n2,d2) \\
\end{array}
$

on a bien :

$\cal{M}(q1 + q2) = (n1,d1) \oplus (n2,d2)$

Dans l'exemple montré plus haut, nous avons calculé 2/3 + 1/3 = 9/9 , ce qui est une représentation possible de 1. Et 1 est bien la somme de 2/3 et 1/3. On vérifie donc cette propriété sur cet exemple.

On voit aussi tout de suite l'inconvénient d'avoir un morphisme non injectif, c'est-à dire qu'on peut avoir :

$f(x) \neq f(y) \quad \text{et} \quad x = y $

C'est le cas du rationnel 1 qui peut se représenter par n'importe quel couple d'entier non nul éguax entre eux : (1,1), (9,9) ...

On peut s'accomoder de cet état. La comparaison *naturelle* de deux couples en comparant numérateur et dénominateur ne fonctionne pas. Par contre, on peut la remplacer par une soustraction des deux couples pour vérifier si cela vaut 0 (c'est à dire un couple d'entier dont le numérateur est nul). Avec cet opérateur de comparaison, on aurait déjà pas mal de possibilités d'utilisation.

Mais nous voyons maintenant comment nous pourrions améliorer cette représentation.




# Retrouver un automorphisme.

Pour trouver une représentation unique d'un rationnel avec deux entiers, il faut que le numérateur et le dénominateur soient réduits pour être premiers entre eux.

 Nous devons donc apprendre à décomposer un entier en produit de facteurs premiers. Pour cela, il faut savoir trouver des nombres premiers. Pour aller vite, nous utilisons ici une liste de nombres premiers. Ceci limite la taille des nombres que nous pourrons décomposer, mais nous nous en contentons pour le moment.

In [1]:
prime_numbers = [2,3,5,7,11,13,17,19,23,29,31,37,41,43,47,53,59,61,67,71,73,
    79,83,89,97,101,103,107,109,113,127,131,137,139,149,151,157,163,167,173,
    179,181,191,193,197,199,211,223,227,229,233,239,241,251,257,263,
    269,271,277,281,283,293,307,311,313,317,331,337,347,349,353,359,
    367,373,379,383,389,397,401,409,419,421,431,433,439,443,449,457,
    461,463,467,479,487,491,499,503,509,521,523,541,547,557,563,569,
    571,577,587,593,599,601,607,613,617,619,631,641,643,647,653,659,
    661,673,677,683,691,701,709,719,727,733,739,743,751,757,761,769,
    773,787,797,809,811,821,823,827,829,839,853,857,859,863,877,881,
    883,887,907,911,919,929,937,941,947,953,967,971,977,983,991,997,
    1009,1013,1019,1021]

def fact (x):
    """Retourne la décomposition de x en liste de ses facteurs premiers."""
    if (x > 1021):
        raise Exception(f"{x} est trop grand pour la liste des nombres premiers disponible.")
    result = []
    for d in prime_numbers:
        while x % d == 0:
            if d > x:
                return result
            elif x % d == 0:
                result.append(d)
                x = x // d   # division entière
                if x == 1:
                    return result
    return result

NUMBER = 765
n_lst = fact(NUMBER)
if NUMBER == 0:
    print("0 n'est pas divisible en produits de facteurs premiers.")
else:
    # on affiche le résultat en même temps qu'on le vérifie
    prod = 1
    for n in n_lst:
        prod = prod * n
    print(f"{prod} se décompose en {n_lst}")

765 se décompose en [3, 3, 5, 17]


Ayant maintenant cette décomposition en produit de facteurs premiers (en liste), nous construisons une fonction réduisant un couple d'entiers en deux entiers premiers entre eux.

In [3]:
def reduce(x):
  """Retourne la forme réduite d'un rationnel."""

  # Traitement des cas triviaux
  num = x[0]
  if num == 0:
    return [0,1]
  elif num == 1:
    return x
  
  denom = x[1]
  if denom == 0:
    raise Exception("Le dénominateur ne peut pas être nul !")
  elif denom == 1:
    return x
  
  dlst = fact(denom)  # décomposition du dénominateur
  if len(dlst) == 1:
    d = dlst[0]
    if num % d == 0:
      return [num // d, 1]
    else:
      return x

  # Boucle réduisant le numérateur  avec les entiers premiers
  # obtenus dans la décomposition du dénominateur.
  for d in dlst:
    if num % d == 0:
      num = num // d
      denom = denom // d
  
  return [num, denom]

print(reduce([765,15]))

[51, 1]


Avec cette fonction de réduction, on peut redéfinir l'addition.

In [5]:
def add2(x , y):
    """Seconde version de l'addition de deux rationnels."""
    return reduce([x[0]*y[1] + y[0]*x[1], x[1]*y[1]])

x1 = [2, 3]
y1 = [1, 3]
s1 = add2(x1,y1)

print(f"Nous avons fait l'opération: {x1} + {y1} = {s1}")

Nous avons fait l'opération: [2, 3] + [1, 3] = [1, 1]


Le résultat obtenu est plus propre. Exerçons nous sur quelques exemples supplémentaires.

In [6]:
x3 = [2, 3]         # = 2 / 3
y3 = [1, 6]         # = 1 / 6
print(add2(x3,y3))

[5, 6]


Avec la fonction précédente, on aurait obtenu.

In [9]:
print(add(x3,y3))

(15, 18)


Le morphisme qui associe un rationnel à un couple d'entiers premiers entre eux est injectif: si deux rationnels sont différents, les couples d'entiers premiers entre eux qui les représentent le sont aussi. Si on limite l'ensemble des couples d'entiers aux couples d'entiers premiers entre eux, alors on peut établir une bijection entre les rationnels et les couples d'entiers premiers entre eux (le second devant toujours être différent de 0). En prenant la loi d'addition des rationnels et celle que nous avons définie pour les couples d'entiers premiers entre eux, cette transformation devient un automorphisme.

# Premier bilan.

La première version de l'addition de deux rationnels peut suffire. Elle reste simple et il faut se souvenir que pour comparer deux rationnels, il ne faut pas utiliser l'égalité membre à membre dans les couples, mais comparer la différence des couples à 0.

Par contre, l'absence de réduction de la représentation du rationnel après chaque opération peut aboutir à une inflation rapide du dénominateur avec des grands nombres entiers, pouvant arriver aux limites de la représentation entière choisie sur l'ordinateur, alors que cela ne se justifie pas. Ceci limite beaucoup l'utilisation de cette méthode.

La deuxième méthode règle ce problème. Nous savons que la représentation de notre rationnel par un couple d'entier premier entre eux existe et est unique, car nous avons avons un automorphisme. Par contre cette représentation requiert une décomposition en produit de facteur premiers. Nous avons utilisé une liste de nombre premiers qui limite donc la taille maximale des entiers. On pourrait imaginer une liste couvrant tous les nombres premiers jusqu'à la limite de représentation du nombre premier ou d'uitiliser un algorithme comme le [crible d'Erathostène](https://www.kartable.fr/ressources/mathematiques/methode/le-crible-derathostene/4672). Mais ces deux méthodes sont lourdes. En informatique, la recherche de nombres premiers (surtout les grands) est un problème tellement long et incommode, qu'on les utilise pour la cryptographie. Une méthode de calcul utilisant des nombres premiers pour quelque chose d'aussi éléméntaire qu'une addition est donc pénalisée et c'est la raison pour laquelle elle n'est pas utilisée par défaut et qu'on préfère approximer les rationnels par des flottants.

# Extension au corps des rationnels.

Jusqu'ici, nous sommes limités à l'ensemble des rationnels muni de l'addition. On peut y ajouter la mutliplication des rationnels et sur notre ensemble de couples de nombres premiers entre eux, on peut définir cette loi de multiplication:  $(n1, d1) \otimes (n2, d2) = (n1 \times n2 , d1 \times d2)$.

On pourra vérifier que l'ensemble des couples avec ces deux lois ainsi définies est un corps et qu'il est relié au corps des nombres rationnels par automorphisme.

Le seul point un peu délicat est peut être la définition d'un inverse, mais qui marche très bien: $ (n, d)^{-1} = (d, n)$

# D'autres solutions ?

Cherchons d'autre structures possibles sur les couples d'entier pour montre qu'il n'est pas si simple d'en trouver une qui soit aussi achevée que celle que nous avons définie.

Imaginons par exemple que nous ayons défini l'addition de deux couples de la façon suivante : $(n1, d1) \oplus (n2, d2) = (n1 + n2 , d1 + d2)$

Pour que cette loi d'addition soit interne, il faut accepter les dénomintateurs nuls car : $(1, 1) \oplus (1,-1) = (2, 0)$

Nous pourrions ranger $(0,0)$ avec les autres couples ayant un numérateur nul et qui sont lié au rationnel $0$ et on pourrait vérifier une structure de groupe (et même de groupe commutatif). 

Par contre, en introduisant la multiplication nous allons au devant de difficultés pour définir un inverse pour tous les couples.

$(0, 1) \otimes (1, 0) = (0 , 0 )$ or $(1,0)$ étant l'inverse de $(0,1)$ nous aurions dû avoir $(1,1)$ comme résultat !

Pour tous les couples dont le dénominateur est $0$, il sera impossible de trouver un autre couple donnant l'équivalent de $(1,1)$ par multiplication.

Cette structure avec cette loi d'addition et de multiplication n'est donc pas un corps. On pourra tout au plus lui donner une structure d'anneau.

# Ce qu'il faut retenir.

Ce cahier a montré comment des rudiments d'algèbre guident et affermissent le raisonnement. Nous avons ici utilisé nos connaissances élémentaires des nombres rationnels pour définir des lois sur des couples d'entiers qui aboutissent à une structure de corps. L'ensemble des couples d'entiers premiers entre eux munis de ces deux lois et lié par automrophisme au corps des rationnels et nous pouvons donc utiliser sur cet ensemble toutes les propriétés algébriques que nous connaissons sur les rationnels.

Bien que rarement nommées, les lois de compositions sont finalement plus déterminantes que les objets eux-mêmes. 

